In [1]:
import os, sys

base_path = os.path.join(os.getcwd(), "..")
print(f"Base Path: {base_path}")
sys.path.append(base_path)

Base Path: /Users/swkim/Documents/coding/thesis/PROMES_colab/notebook/..


# Data Generation

## 1. 1-label data generation

In [2]:
# Load gym environment
import gym
from kube_sim_gym import *
from kube_sim_gym.envs.sim_kube_env import SimKubeEnv

In [3]:
from kube_hr_scheduler.scheduler.sim_hr_scheduler import SimHrScheduler
from kube_hr_scheduler.strategies.model.default import Model

In [4]:
env = gym.make('SimKubeEnv-v0')
scheduler = SimHrScheduler(env, 'default.py')

In [ ]:
# Data generation
# Data consists of state, scheduler decision
data_size = 1000000
data_path = os.path.join(base_path, "dataset", "data_1.csv")
with open(data_path, 'w') as f:
    for i in range(data_size):
        state = list(env.random_state_gen())
        action = scheduler.decision(env)
        data = state + [action]
        f.write(','.join(list(map(str,data))) + '\n')

## 2. 6-confidence data generation

In [7]:
# Load gym environment
import gym
from kube_sim_gym import *
from kube_sim_gym.envs.sim_kube_env import SimKubeEnv

In [8]:
from kube_hr_scheduler.scheduler.sim_hr_scheduler import SimHrScheduler
from kube_hr_scheduler.strategies.model.default import Model

In [9]:
env = gym.make('SimKubeEnv-v0')
scheduler = SimHrScheduler(env, 'default.py')

In [ ]:
# Data generation
# Data consists of state, scheduler decision
data_size = 1000000
data_path = os.path.join(base_path, "dataset", "data_5.csv")
with open(data_path, 'w') as f:
    for i in range(data_size):
        state = list(env.random_state_gen())
        confidence = scheduler.model.get_confidence(env)
        data = state + confidence.tolist()
        f.write(','.join(list(map(str,data))) + '\n')

## 3. 1-confidence data generation

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from kube_mm_scheduler.model.net3 import Net3

net3 = Net3()

In [4]:
# predict the score for each action
def predict_score(data, model=net3):
    model.eval()
    if not torch.is_tensor(data):
        data = torch.tensor(data, dtype=torch.float32)
    data10 = data[:, :-2]
    data2 = data[:, -2:]
    output = model(data10, data2)
    # Round up at the 4th decimal point
    output = torch.round(output * 10000) / 10000
    # # Softmax
    # output = F.softmax(output, dim=1)
    # print(output)
    return output

In [5]:
# Read from data_1.csv and formulate data_3.csv which has additional selected node information
src_path = os.path.join(base_path, "dataset", "data_1.csv")
with open(src_path, 'r') as f:
    data_path = os.path.join(base_path, "dataset", "data_3.csv")
    with open(data_path, 'w') as g:
        for line in f.readlines():
            line = line.strip()
            line = line.split(',')
            node_state = line[:-3]
            pod_quota = line[-3:-1]

            # Confidence
            flt_state = list(map(float, node_state+pod_quota))
            # print(flt_state)
            confidence = predict_score([flt_state])

            for i in range(5):
                selected_node_state = node_state[i*2:i*2+2]
                new_state = node_state + pod_quota + selected_node_state
                conf = confidence[0].tolist()[i+1]
                # Takes the 4th decimal point
                conf = round(conf, 4)
                
                new_data = new_state + [conf]
                g.write(','.join(list(map(str,new_data))) + '\n')

KeyboardInterrupt: 

# 4. 6-reward data generation

In [2]:
# Load gym environment
import gym
from kube_sim_gym import *
from kube_sim_gym.envs.sim_kube_env import SimKubeEnv

In [3]:
from kube_hr_scheduler.scheduler.sim_hr_scheduler import SimHrScheduler
from kube_hr_scheduler.strategies.model.default import Model

In [4]:
env = gym.make('SimKubeEnv-v0', reward_file='train_dynamic.py')
scheduler = SimHrScheduler(env, 'default.py')

Base Path: /Users/swkim/Documents/coding/thesis/PROMES_colab/notebook/..


In [5]:
import tqdm

data_size = 1000000
data_path = os.path.join(base_path, "dataset", "data_4.csv")

with open(data_path, 'w') as f:
    for i in tqdm.tqdm(range(data_size)):
        env.reset()
        state = list(env.random_state_gen())
        # print(f"State: {state}")

        env0 = env.duplicate() # For action 0
        env1 = env.duplicate() # For action 1
        env2 = env.duplicate() # For action 2
        env3 = env.duplicate() # For action 3
        env4 = env.duplicate() # For action 4
        env5 = env.duplicate() # For action 5

        state0, rew0, _, _ = env0.step(0, True)
        # print(f"Action 0")
        # print(f"State0: {state0}")
        # print("====================")
        state1, rew1, _, _ = env1.step(1, True)
        # print(f"Action 1")
        # print(f"State1: {state1}") 
        # print("====================")
        state2, rew2, _, _ = env2.step(2, True)
        # print(f"Action 2")
        # print(f"State2: {state2}")
        # print("====================")
        state3, rew3, _, _ = env3.step(3, True)
        # print(f"Action 3")
        # print(f"State3: {state3}")
        # print("====================")
        state4, rew4, _, _ = env4.step(4, True)
        # print(f"Action 4")
        # print(f"State4: {state4}")
        # print("====================")
        state5, rew5, _, _ = env5.step(5, True)
        # print(f"Action 5")
        # print(f"State5: {state5}")

        reward = [rew0, rew1, rew2, rew3, rew4, rew5]
        # Round up at the 4th decimal point
        reward = list(map(lambda x: round(x, 4), reward))
        
        f.write(','.join(list(map(str,state))) + '  ,  ' + ','.join(list(map(str,reward))) + '\n')

 16%|█▌        | 159051/1000000 [1:36:41<17:43:41, 13.18it/s]

In [ ]:
import numpy as np

np.random.random()

0.1243402113209463